In [1]:
import nest

## Projection object

Introducing the Projection object, representing projection between two populations with a certain rule. The projection object is added to a buffer, and held until calling `projections.build_network()`.

### `OneToOne`

In [2]:
nest.ResetKernel()

n = nest.Create('iaf_psc_alpha', 5)

projection = nest.projections.OneToOne(source=n, target=n)
nest.projections.Connect(projection)
nest.projections.build_network()

conns = nest.GetConnections()
print(conns)

*--------*----------------*
| source | 1, 2, 3, 4, 5, |
*--------*----------------*
| target | 1, 2, 3, 4, 5, |
*--------*----------------*


### `FixedIndegree`
Other projections can have additional arguments.

In [3]:
nest.ResetKernel()

n = nest.Create('iaf_psc_alpha', 5)

projection = nest.projections.FixedIndegree(source=n, target=n, indegree=5)
nest.projections.Connect(projection)
nest.projections.build_network()

conns = nest.GetConnections()
print(conns)

*--------*----------------------------------------------------------------------------*
| source | 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 5, 5, |
*--------*----------------------------------------------------------------------------*
| target | 3, 1, 3, 1, 2, 2, 2, 2, 3, 1, 4, 4, 5, 3, 5, 5, 5, 1, 2, 3, 4, 4, 5, 1, 4, |
*--------*----------------------------------------------------------------------------*


### `FixedOutdegree`

In [4]:
nest.ResetKernel()

n = nest.Create('iaf_psc_alpha', 5)

projection = nest.projections.FixedOutdegree(source=n, target=n, outdegree=5)
nest.projections.Connect(projection)
nest.projections.build_network()

conns = nest.GetConnections()
print(conns)

*--------*----------------------------------------------------------------------------*
| source | 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, |
*--------*----------------------------------------------------------------------------*
| target | 3, 2, 4, 2, 1, 1, 3, 5, 4, 3, 4, 5, 5, 5, 2, 2, 2, 4, 5, 5, 5, 4, 1, 5, 2, |
*--------*----------------------------------------------------------------------------*


### `FixedTotalNumber`

In [5]:
nest.ResetKernel()
n = nest.Create('iaf_psc_alpha', 5)

projection = nest.projections.FixedTotalNumber(source=n, target=n, N=10, 
                                   allow_autapses=True,
                                   allow_multapses=True,
                                   syn_spec={'weight': 0.5})
nest.projections.Connect(projection)
nest.projections.build_network()

conns = nest.GetConnections()
print(conns)
print(conns.weight)

*--------*-------------------------------*
| source | 1, 1, 2, 2, 2, 2, 2, 3, 4, 4, |
*--------*-------------------------------*
| target | 2, 4, 1, 5, 2, 5, 4, 4, 2, 2, |
*--------*-------------------------------*
[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]


### `PairwiseBernoulli`

In [6]:
nest.ResetKernel()
n = nest.Create('iaf_psc_alpha', 10)

projection = nest.projections.PairwiseBernoulli(source=n, target=n, p=0.5)
nest.projections.Connect(projection)
nest.projections.build_network()

conns = nest.GetConnections()
print(conns)

*--------*----------------------------------------------------------------------------------------------------*
| source | 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, ... 8, 8, 8, 9, 9, 9, 9, 9, 9, 9, 10, 10, 10, 10, 10, |
*--------*----------------------------------------------------------------------------------------------------*
| target | 10, 6, 1, 5, 4, 9, 3, 2, 7, 2, 1, 8, 9, 2, 10, ... 1, 3, 9, 1, 7, 5, 8, 3, 2, 9, 6, 1, 8, 3, 10, |
*--------*--------------------------------------------------------------------------------------------------*


## Multiple projections

In [7]:
nest.ResetKernel()

N = 10
IN_A = 2
IN_B = 5
n = nest.Create('iaf_psc_alpha', N)
nest.projections.Connect(nest.projections.FixedIndegree(source=n, target=n, indegree=IN_A))
nest.projections.Connect(nest.projections.FixedIndegree(source=n, target=n, indegree=IN_B))
nest.projections.build_network()

conns = nest.GetConnections()
print(conns)

*--------*---------------------------------------------------------------------------------------------------*
| source | 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, ... 7, 8, 8, 8, 8, 8, 8, 8, 9, 9, 9, 10, 10, 10, 10, |
*--------*---------------------------------------------------------------------------------------------------*
| target | 9, 4, 3, 6, 6, 3, 7, 2, 8, 6, 1, 5, 5, 5, 8, ... 9, 10, 6, 1, 10, 4, 3, 3, 10, 9, 2, 8, 6, 4, 4, |
*--------*--------------------------------------------------------------------------------------------------*


## Using `Connect()`

In [8]:
nest.ResetKernel()

n = nest.Create('iaf_psc_alpha', 5)
nest.projections.Connect_immediately(nest.projections.OneToOne(source=n, target=n))

conns = nest.GetConnections()
print(conns)


*--------*----------------*
| source | 1, 2, 3, 4, 5, |
*--------*----------------*
| target | 1, 2, 3, 4, 5, |
*--------*----------------*


### List of multiple Projections

In [10]:
nest.ResetKernel()

N = 5
IN_A = 2
IN_B = 4
n = nest.Create('iaf_psc_alpha', N)
nest.projections.Connect_immediately([nest.projections.FixedIndegree(source=n, target=n, indegree=IN_A),
                                      nest.projections.FixedIndegree(source=n, target=n, indegree=IN_B)])

conns = nest.GetConnections()
print(conns)

*--------*-------------------------------------------------------------------------------------------*
| source | 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 5, 5, |
*--------*-------------------------------------------------------------------------------------------*
| target | 1, 1, 5, 4, 1, 5, 4, 4, 5, 5, 3, 1, 5, 2, 1, 2, 5, 3, 2, 4, 3, 3, 4, 4, 3, 1, 3, 2, 2, 2, |
*--------*-------------------------------------------------------------------------------------------*
